## 로지스틱 회귀분석 연습

### 문제
암 데이터에서 diagnosis를 예측하는 로지스틱 회귀분석을 해보세요. 독립변수로는 area_worst, smoothness_worst, symmetry_worst, fractal_dimension_worst 4가지를 사용하세요.
데이터 출처: http://doc.mindscale.kr/ngv/cancer.xlsx

1. area_worst의 기울기는 얼마입니까?
2. 위 모형의 정확도는 얼마입니까? (M을 양성으로 간주, 소수점 셋째 자리에서 반올림)
3. 위 모형의 정밀도는 얼마입니까? (M을 양성으로 간주, 소수점 셋째 자리에서 반올림)
4. 위 모형의 재현도는 얼마입니까? (M을 양성으로 간주, 소수점 셋째 자리에서 반올림)
5. 위의 모형을 모형 1이라 하고, 여기에서 fractal_dimension_worst를 제외한 모형을 모형 2라고 할 때 어떤 모형이 더 로그우도가 높습니까?
6. 모형1과 모형2를 비교할 때, 어떤 모형이 더 나은 모형이라고 할 수 있습니까?

In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_excel('cancer.xlsx')

In [33]:
df

,diagnosis,texture_mean,smoothness_mean,concave_points_mean,symmetry_mean,perimeter_se,compactness_se,concavity_se,concave_points_se,area_worst,smoothness_worst,symmetry_worst,fractal_dimension_worst
0,M,10.38,0.11840,0.14710,0.2419,8.589,0.04904,0.05373,0.01587,2019.0,0.16220,0.4601,0.11890
1,M,17.77,0.08474,0.07017,0.1812,3.398,0.01308,0.01860,0.01340,1956.0,0.12380,0.2750,0.08902
2,M,21.25,0.10960,0.12790,0.2069,4.585,0.04006,0.03832,0.02058,1709.0,0.14440,0.3613,0.08758
3,M,20.38,0.14250,0.10520,0.2597,3.445,0.07458,0.05661,0.01867,567.7,0.20980,0.6638,0.17300
4,M,14.34,0.10030,0.10430,0.1809,5.438,0.02461,0.05688,0.01885,1575.0,0.13740,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,22.39,0.11100,0.13890,0.1726,7.673,0.02891,0.05198,0.02454,2027.0,0.14100,0.2060,0.07115
565,M,28.25,0.09780,0.09791,0.1752,5.203,0.02423,0.03950,0.01678,1731.0,0.11660,0.2572,0.06637
566,M,28.08,0.08455,0.05302,0.1590,3.425,0.03731,0.04730,0.01557,1124.0,0.11390,0.2218,0.07820
567,M,29.33,0.11780,0.15200,0.2397,5.772,0.06158,0.07117,0.01664,1821.0,0.16500,0.4087,0.12400


In [34]:
# 더미코딩이 되어 있지 않은 종속변수 diagnosis를 0, 1 범주로 변환
df['result'] = df['diagnosis'].map({'B':0, 'M':1})
df

,diagnosis,texture_mean,smoothness_mean,concave_points_mean,symmetry_mean,perimeter_se,compactness_se,concavity_se,concave_points_se,area_worst,smoothness_worst,symmetry_worst,fractal_dimension_worst,result
0,M,10.38,0.11840,0.14710,0.2419,8.589,0.04904,0.05373,0.01587,2019.0,0.16220,0.4601,0.11890,1
1,M,17.77,0.08474,0.07017,0.1812,3.398,0.01308,0.01860,0.01340,1956.0,0.12380,0.2750,0.08902,1
2,M,21.25,0.10960,0.12790,0.2069,4.585,0.04006,0.03832,0.02058,1709.0,0.14440,0.3613,0.08758,1
3,M,20.38,0.14250,0.10520,0.2597,3.445,0.07458,0.05661,0.01867,567.7,0.20980,0.6638,0.17300,1
4,M,14.34,0.10030,0.10430,0.1809,5.438,0.02461,0.05688,0.01885,1575.0,0.13740,0.2364,0.07678,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,22.39,0.11100,0.13890,0.1726,7.673,0.02891,0.05198,0.02454,2027.0,0.14100,0.2060,0.07115,1
565,M,28.25,0.09780,0.09791,0.1752,5.203,0.02423,0.03950,0.01678,1731.0,0.11660,0.2572,0.06637,1
566,M,28.08,0.08455,0.05302,0.1590,3.425,0.03731,0.04730,0.01557,1124.0,0.11390,0.2218,0.07820,1
567,M,29.33,0.11780,0.15200,0.2397,5.772,0.06158,0.07117,0.01664,1821.0,0.16500,0.4087,0.12400,1


In [35]:
from statsmodels.formula.api import logit

In [36]:
df.columns

Index(['diagnosis', 'texture_mean', 'smoothness_mean', 'concave_points_mean',
       'symmetry_mean', 'perimeter_se', 'compactness_se', 'concavity_se',
       'concave_points_se', 'area_worst', 'smoothness_worst', 'symmetry_worst',
       'fractal_dimension_worst', 'result'],
      dtype='object')

In [37]:
# 아래 4개를 독립변수로 두는 로지스틱 회귀분석
res1 = logit('result ~ area_worst + smoothness_worst + symmetry_worst + fractal_dimension_worst', data=df).fit()
res1.summary()

Optimization terminated successfully.
         Current function value: 0.112036
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 result   No. Observations:                  569
Model:                          Logit   Df Residuals:                      564
Method:                           MLE   Df Model:                            4
Date:                Wed, 28 Oct 2020   Pseudo R-squ.:                  0.8303
Time:                        12:39:17   Log-Likelihood:                -63.749
converged:                       True   LL-Null:                       -375.72
Covariance Type:            nonrobust   LLR p-value:                1.019e-133
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 -26.2311      3.150     -8.327      0.000     -32.405     -20.057
area_worst                  0.0147      0.002      8.244      0.000       0.011       0.018
smoothness_worst           68.3573     16.590      4.120      0.000      35.842     100.873
symmetry_worst             10.6379      4.559      2.334      0.020       1.703      19.573
fractal_dimension_worst    17.9945     19.247      0.935      0.350     -19.728      55.717
===========================================================================================

Possibly complete quasi-separation: A fraction 0.20 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [38]:
# fractal_dimension_worst를 제외한 3개를 독립변수로 두는 로지스틱 회귀분석, res1(모형1)과 비교
res2 = logit('result ~ area_worst + smoothness_worst + symmetry_worst', data=df).fit()
res2.summary()

Optimization terminated successfully.
         Current function value: 0.112809
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 result   No. Observations:                  569
Model:                          Logit   Df Residuals:                      565
Method:                           MLE   Df Model:                            3
Date:                Wed, 28 Oct 2020   Pseudo R-squ.:                  0.8292
Time:                        12:39:18   Log-Likelihood:                -64.189
converged:                       True   LL-Null:                       -375.72
Covariance Type:            nonrobust   LLR p-value:                1.008e-134
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -26.4085      3.145     -8.398      0.000     -32.572     -20.245
area_worst           0.0148      0.002      8.272      0.000       0.011       0.018
smoothness_worst    76.7460     14.452      5.311      0.000      48.421     105.071
symmetry_worst      12.3211      4.223      2.918      0.004       4.044      20.598
====================================================================================

Possibly complete quasi-separation: A fraction 0.21 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

### 로그우도: 더 복잡한 모형이 더 높게 나옴

In [58]:
res1.llf > res2.llf

True

In [39]:
# 어떤 모형이 더 나은지 비교(로그우도는 모형 1이 더 높음)
print(res1.aic)
print(res1.bic)
print(res1.prsquared)
print('-'*20)
print(res2.aic)
print(res2.bic)
print(res2.prsquared)

137.49732244744993
159.21672461808157
0.8303293389753541
--------------------
136.37712254915456
153.7526442856599
0.8291585201537486


In [60]:
res1.aic > res2.aic

True

In [61]:
res1.bic > res2.bic

True

aic, bic로만 봤을 때는 모형2가 더 나음(값이 더 낮음)

In [40]:
# res1의 정확도, 정밀도, 재현도 구하기 -- 문턱값은 0.5
prob = res1.predict(df)
prob

0      1.000000
1      1.000000
2      0.999999
3      0.998687
4      0.999964
         ...   
564    1.000000
565    0.999985
566    0.864486
567    1.000000
568    0.000007
Length: 569, dtype: float64

In [44]:
문턱값 = 0.5
prediction = np.where(prob > 문턱값, 1, 0)

### 혼동 행렬(confusion matrix) 만들기
- 새로운 데이터를 넣어 혼동행렬을 만드는 게 좋다.

아래는 행렬 생김새 참고
<br>
True Negative  | False Positive
<br>
False Negative | True Positive

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df['result'], prediction)

array([[349,   8],
       [ 16, 196]])

In [28]:
# 정밀도 구하기
round((196) / (8 + 196), 3)

0.961

In [46]:
# 정확도 구하기
round((349 + 196) / (349 + 8 + 16 + 196), 3)

0.958

In [29]:
# 재현도 구하기
round((196) / (16 + 196), 3)

0.925

In [54]:
# 이런 함수를 써서 바로 구할 수도 있음 ^ ^
from sklearn.metrics import accuracy_score, precision_score, recall_score
accuracy_score(df['result'], prediction)

0.9578207381370826

In [55]:
precision_score(df['result'], prediction)

0.9607843137254902

In [56]:
recall_score(df['result'], prediction)

0.9245283018867925

### 해석시 참고(from 재명님)
- 로그우도 = 기울기에서 우리가 가진 데이터가 나올 확률, 즉 기존 데이터를 잘 예측했느냐에 관한.. 잠깐 이거 R^2 ?
- 모형의 복잡도가 높으면 로그우도가 높아짐
- AIC = 로그우도에서 복잡도를 보정
- 로그우도(R제곱과 비슷)는 기존의 데이터를 얼마나 잘 예측했는가를 보는 것, aic, bic는 앞으로 모형을 얼마나 잘 예측하는가

----

## 다중공선성 실습

In [62]:
car = pd.read_excel('small_used_car.xlsx')

In [63]:
car

,mileage,model,price,year,my_car_damage,other_car_damage
0,63608,K3,970,2017,0,564596
1,69336,K3,1130,2015,1839700,1140150
2,36000,K3,1380,2016,446520,2244910
3,19029,K3,1390,2017,889000,4196110
4,97090,K3,760,2015,2339137,2029570
...,...,...,...,...,...,...
269,235000,Avante,390,2007,1473730,507260
270,140000,Avante,430,2009,14399366,4592589
271,96757,Avante,390,2006,303080,0
272,113853,Avante,390,2008,320780,1857718


In [64]:
from statsmodels.formula.api import ols

In [68]:
ols('price ~ mileage + model + year + my_car_damage + other_car_damage', data=car).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     193.2
Date:                Wed, 28 Oct 2020   Prob (F-statistic):           1.14e-86
Time:                        15:50:42   Log-Likelihood:                -1770.2
No. Observations:                 274   AIC:                             3552.
Df Residuals:                     268   BIC:                             3574.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept        -1.523e+05   9586.614    -15.885      0.000   -1.71e+05   -1.33e+05
model[T.K3]        -32.7958     22.205     -1.477      0.141     -76.514      10.923
mileage             -0.0024      0.000     -8.958      0.000      -0.003      -0.002
year                76.1874      4.755     16.022      0.000      66.825      85.550
my_car_damage    -1.619e-05   3.31e-06     -4.894      0.000   -2.27e-05   -9.68e-06
other_car_damage -6.073e-06   2.14e-06     -2.836      0.005   -1.03e-05   -1.86e-06
==============================================================================
Omnibus:                       11.358   Durbin-Watson:                   1.553
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               14.249
Skew:                           0.347   Prob(JB):                     0.000805
Kurtosis:                       3.875   Cond. No.                     5.46e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.46e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [69]:
ols('mileage ~ model + year + my_car_damage + other_car_damage', data=car).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                mileage   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     34.36
Date:                Wed, 28 Oct 2020   Prob (F-statistic):           3.62e-23
Time:                        15:51:17   Log-Likelihood:                -3255.1
No. Observations:                 274   AIC:                             6520.
Df Residuals:                     269   BIC:                             6538.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         1.956e+07    1.8e+06     10.861      0.000     1.6e+07    2.31e+07
model[T.K3]       -160.5713   5003.433     -0.032      0.974      -1e+04    9690.298
year             -9677.1395    894.403    -10.820      0.000   -1.14e+04   -7916.218
my_car_damage        0.0005      0.001      0.679      0.498      -0.001       0.002
other_car_damage    -0.0011      0.000     -2.319      0.021      -0.002      -0.000
==============================================================================
Omnibus:                       45.374   Durbin-Watson:                   1.670
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               86.746
Skew:                           0.871   Prob(JB):                     1.46e-19
Kurtosis:                       5.137   Cond. No.                     4.56e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.56e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [71]:
1/(1-0.338)

1.5105740181268883